<a href="https://colab.research.google.com/github/pedromedeiros1418/Resid-ncia/blob/main/modelo_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Bibliotecas de manipulação e visualização de dados
import numpy as np
import pandas as pd
from scipy.io import arff
import matplotlib.pyplot as plt

# Classes do modelo
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.tree import DecisionTreeClassifier  
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

# Funções de avaliação dos modelos
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, make_scorer, roc_auc_score, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold

#Load the data using "arff.loadarff" then convert it to dataframe
data = arff.loadarff('dataset_30_page-blocks.arff')

In [ ]:
df = pd.DataFrame(data[0])
df.info()

In [ ]:
#tratamento de dados da coluna class - object para int
df['class'] = df['class'].astype(int)

#entradas
X = df.iloc[:,:-1].values
#rotulos
y = df.iloc[:,10].values

In [ ]:
model = GaussianNB()

#a variavel var_smoothing adiciona artificialmente um valor à variação da distribuição
#suaviza a curva e representa mais amostrar que estão mais distantes da média
#de distribuição

parametros = {'var_smoothing': np.logspace(0,-9, num=100)}

In [ ]:
# Random Trails
# Ter o mesmo resultado independentemente de quantas vezes seja executado
NUM_TRIALS = 5

for i in range(NUM_TRIALS):

  #Variáveis para relatório de classificação
  cls_original = []
  cls_predita = []

  #utilizado para dividir reparticionar o dataset para o GridSearchCV
  inner_cv = KFold(n_splits=10, shuffle=True, random_state=i) 

  #utilizado para reparticionar o X_train para avaliação do cross-validate
  outer_cv = KFold(n_splits=10, shuffle=True, random_state=i)

  #customização do relatório utilizando acurácia
  def classification_report_with_accuracy_score(y_true, y_pred):
    cls_original.extend(y_true)
    cls_predita.extend(y_pred)
    return accuracy_score(y_true, y_pred) # return accuracy score

  #GridSearchCV para encontrar os melhores parâmetros para o modelo
  clf = GridSearchCV(model, param_grid=parametros, cv=inner_cv)
  clf.fit(X,y)

  print(clf.best_params_)

  #Cross-validation aninhado com gridSearch - utiliza o melhor parâmetro para avaliar 
  #Reparticionando em 10-fold
  scores = cross_val_score(clf, X, y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

#Valores médios no relatório de classificação para todas as folds em uma validação cruzada 10-fold
print(classification_report(cls_original, cls_predita, digits=4))

In [ ]:
print(confusion_matrix(cls_original, cls_predita))

In [ ]:
roc_auc_score(y, clf.predict_proba(X), multi_class='ovr')